# An Analysis of Yield Tokenization and Arbitrage Opportunities for Liquid Restaking Tokens on Pendle

- What are the drivers of the fixed yields? Is there a correlation between yields and the token price in which the reward are being distributed?
- How correlated are each LRTs between each other?
- How are the various tenors correlated?
- Is there also a liquidity factor: i,e llliquid pools outperform? Why do some LRT offer more yield than others because that should not exist?
- Are there any possible arbitrage strategies that can be created?

YT/PT data from pendle.
Underlying data from Yahoo finance.

In [20]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from utils import prepare_token_data, prepare_price_data
import api.address_constants as ac
from datetime import datetime

sns.set(style='whitegrid')

# Data paths
eETH_underlying_path = 'data/eETH/EETH-USD.csv'
eETH_tvl_path = 'data/eETH/eETH_tvl_data.csv'
etherfi_eETH_yt_jun_path = 'data/eETH/etherfi_eETH_Jun_yt_ohlcv_data.csv'
etherfi_eETH_pt_jun_path = 'data/eETH/etherfi_eETH_Jun_pt_ohlcv_data.csv'
etherfi_eETH_yt_sep_path = 'data/eETH/etherfi_eETH_Sep_yt_ohlcv_data.csv'
etherfi_eETH_pt_sep_path = 'data/eETH/etherfi_eETH_Sep_pt_ohlcv_data.csv'
etherfi_eETH_yt_dec_path = 'data/eETH/etherfi_eETH_Dec_yt_ohlcv_data.csv'
etherfi_eETH_pt_dec_path = 'data/eETH/etherfi_eETH_Dec_pt_ohlcv_data.csv'
zircuit_eETH_yt_jun_path = 'data/eETH/zircuit_eETH_Jun_yt_ohlcv_data.csv'
zircuit_eETH_pt_jun_path = 'data/eETH/zircuit_eETH_Jun_pt_ohlcv_data.csv'

ezETH_underlying_path = 'data/ezETH/EZETH-USD.csv'
ezETH_yt_sep_path = 'data/ezETH/ezETH_Sep_yt_ohlcv_data.csv'
ezETH_pt_sep_path = 'data/ezETH/ezETH_Sep_pt_ohlcv_data.csv'
ezETH_yt_dec_path = 'data/ezETH/ezETH_Dec_yt_ohlcv_data.csv'
ezETH_pt_dec_path = 'data/ezETH/ezETH_Dec_pt_ohlcv_data.csv'

pufETH_underlying_path = 'data/pufETH/PUFETH-USD.csv'
pufETH_yt_jun_path = 'data/pufETH/pufETH_Jun_yt_ohlcv_data.csv'
pufETH_pt_jun_path = 'data/pufETH/pufETH_Jun_pt_ohlcv_data.csv'
pufETH_yt_sep_path = 'data/pufETH/pufETH_Sep_yt_ohlcv_data.csv'
pufETH_pt_sep_path = 'data/pufETH/pufETH_Sep_pt_ohlcv_data.csv'

uniETH_underlying_path = 'data/uniETH/UNIETH-USD.csv'
uniETH_yt_jun_path = 'data/uniETH/uniETH_Jun_yt_ohlcv_data.csv'
uniETH_pt_jun_path = 'data/uniETH/uniETH_Jun_pt_ohlcv_data.csv'
uniETH_yt_sep_path = 'data/uniETH/uniETH_Sep_yt_ohlcv_data.csv'
uniETH_pt_sep_path = 'data/uniETH/uniETH_Sep_pt_ohlcv_data.csv'

# Load the data
etherfi_eETH_jun_df = prepare_token_data(eETH_underlying_path, etherfi_eETH_yt_jun_path, etherfi_eETH_pt_jun_path, ac.etherfi_eETH_Jun_start_date, ac.expiry_date_jun)
etherfi_eETH_sep_df = prepare_token_data(eETH_underlying_path, etherfi_eETH_yt_sep_path, etherfi_eETH_pt_sep_path, ac.etherfi_eETH_Sep_start_date, ac.expiry_date_sep)
etherfi_eETH_dec_df = prepare_token_data(eETH_underlying_path, etherfi_eETH_yt_dec_path, etherfi_eETH_pt_dec_path, ac.etherfi_eETH_Dec_start_date, ac.expiry_date_dec)
zircuit_eETH_jun_df = prepare_token_data(eETH_underlying_path, zircuit_eETH_yt_jun_path, zircuit_eETH_pt_jun_path, ac.zircuit_eETH_Jun_start_date, ac.expiry_date_jun)

ezETH_sep_df = prepare_token_data(ezETH_underlying_path, ezETH_yt_sep_path, ezETH_pt_sep_path, ac.ezETH_Sep_start_date, ac.expiry_date_sep)
ezETH_dec_df = prepare_token_data(ezETH_underlying_path, ezETH_yt_dec_path, ezETH_pt_dec_path, ac.ezETH_Dec_start_date, ac.expiry_date_dec)

pufETH_jun_df = prepare_token_data(pufETH_underlying_path, pufETH_yt_jun_path, pufETH_pt_jun_path, ac.pufETH_Jun_start_date, ac.expiry_date_jun)
pufETH_sep_df = prepare_token_data(pufETH_underlying_path, pufETH_yt_sep_path, pufETH_pt_sep_path, ac.pufETH_Sep_start_date, ac.expiry_date_sep)

uniETH_jun_df = prepare_token_data(uniETH_underlying_path, uniETH_yt_jun_path, uniETH_pt_jun_path, ac.uniETH_Jun_start_date, ac.expiry_date_jun)
uniETH_sep_df = prepare_token_data(uniETH_underlying_path, uniETH_yt_sep_path, uniETH_pt_sep_path, ac.uniETH_Sep_start_date, ac.expiry_date_sep)

etherfi_eETH_jun_df


,pt_open,yt_open,underlying_open
2024-03-15,195.8840,3666.3952,3860.714355
2024-03-16,182.4901,3534.3381,3687.615967
2024-03-17,177.6619,3320.8564,3483.889648
2024-03-18,227.3860,3403.7633,3610.342285
2024-03-19,203.5317,3302.4109,3485.050537
...,...,...,...
2024-06-10,30.6050,3673.0982,3690.282227
2024-06-11,26.3883,3636.7638,3663.812256
2024-06-12,22.1899,3474.3848,3491.346680
2024-06-13,22.8114,3534.0871,3552.050781


### What are the drivers of fixed yields

In [21]:
def calculate_implied_apy(prices_df, expiry_date):
    # Calculate the remaining days to expiry
    prices_df['days_to_expiry'] = (expiry_date - prices_df.index).days

    # Convert PT and YT prices from USD to ETH using the underlying price
    prices_df['pt_open_eth'] = prices_df['pt_open'] / prices_df['underlying_open']
    prices_df['yt_open_eth'] = prices_df['yt_open'] / prices_df['underlying_open']

    # Calculate the implied APY using ETH prices
    prices_df['implied_apy_eth'] = (1 + prices_df['yt_open_eth'] / prices_df['pt_open_eth']) ** (365 / prices_df['days_to_expiry']) - 1
    prices_df['implied_apy_usd'] = (1 + prices_df['yt_open'] / prices_df['pt_open']) ** (365 / prices_df['days_to_expiry']) - 1

    return prices_df

def calculate_fixed_yield(df):
    df['fixed_yield'] = (df['underlying_open'] / df['pt_open']) ** (1 / (ac.expiry_date_jun - ac.etherfi_eETH_Jun_start_date).days) - 1
    return df

In [22]:
etherfi_eETH_jun_df = calculate_fixed_yield(etherfi_eETH_jun_df)
etherfi_eETH_sep_df = calculate_fixed_yield(etherfi_eETH_sep_df)
etherfi_eETH_dec_df = calculate_fixed_yield(etherfi_eETH_dec_df)

etherfi_eETH_jun_df = calculate_implied_apy(etherfi_eETH_jun_df, ac.expiry_date_jun)
etherfi_eETH_sep_df = calculate_implied_apy(etherfi_eETH_sep_df, ac.expiry_date_sep)
etherfi_eETH_dec_df = calculate_implied_apy(etherfi_eETH_dec_df, ac.expiry_date_dec)

# Combine data for correlation analysis
etherfi_combined_df = pd.concat([etherfi_eETH_jun_df.assign(maturity='Jun'),
                         etherfi_eETH_sep_df.assign(maturity='Sep'),
                         etherfi_eETH_dec_df.assign(maturity='Dec')])

# Convert dates to ordinal for PCA
etherfi_combined_df['date_ordinal'] = etherfi_combined_df.index.map(datetime.toordinal)

etherfi_combined_df

,pt_open,yt_open,underlying_open,fixed_yield,days_to_expiry,pt_open_eth,yt_open_eth,implied_apy_eth,implied_apy_usd,maturity,date_ordinal
2024-03-15,195.8840,3666.3952,3860.714355,0.029079,104,0.050738,0.949668,35026.317969,35026.317969,Jun,738960
2024-03-16,182.4901,3534.3381,3687.615967,0.029326,103,0.049487,0.958434,43495.321013,43495.321013,Jun,738961
2024-03-17,177.6619,3320.8564,3483.889648,0.029029,102,0.050995,0.953204,42807.242272,42807.242272,Jun,738962
2024-03-18,227.3860,3403.7633,3610.342285,0.026942,101,0.062982,0.942781,22308.432158,22308.432158,Jun,738963
2024-03-19,203.5317,3302.4109,3485.050537,0.027688,100,0.058401,0.947593,32510.013326,32510.013326,Jun,738964
...,...,...,...,...,...,...,...,...,...,...,...
2024-06-10,199.8342,3503.8690,3690.282227,0.028435,199,0.054151,0.949485,210.673720,210.673720,Dec,739047
2024-06-11,193.2681,3469.8840,3663.812256,0.028694,198,0.052751,0.947069,225.645149,225.645149,Dec,739048
2024-06-12,180.7495,3315.8252,3491.346680,0.028880,197,0.051771,0.949727,240.957167,240.957167,Dec,739049
2024-06-13,190.2599,3366.6386,3552.050781,0.028543,196,0.053563,0.947801,232.486577,232.486577,Dec,739050
